In [15]:
import random
import pandas as pd
from transformers import pipeline

In [16]:
# pip install accelerate  # нужен для device_map='auto'

# Генерация синтетических данных

Сгенерируем синтетические данные из заготовленных предложений, подставляю симптом или список симптомов.

In [17]:
symptoms = {
    "vomiting": "рвота",
    "breathlessness": "одышка",
    "sweating": "потливость",
    "headache": "головная боль",
    "chest_pain": "боль в груди",
    "dizziness": "головокружение",
    "loss_of_balance": "потеря равновесия",
    "lack_of_concentration": "отсутствие концентрации"
}

single_symptom_phrases = [
    "я сегодня испытываю {symptom}.",
    "у меня {symptom}, и мне это не нравится.",
    "похоже, что у меня {symptom}.",
    "в последнее время я замечаю {symptom}.",
    "{symptom} меня беспокоит уже несколько дней.",
    "я не могу сосредоточиться из-за {symptom}.",
    "кажется, я чувствую {symptom}.",
    "проблемы с {symptom} не покидают меня.",
    "мне кажется, у меня {symptom}.",
    "уже который день у меня {symptom}.",
    "мой врач сказал, что у меня {symptom}.",
    "меня беспокоит {symptom}.",
    "как долго продлится {symptom}?",
    "не знаю, что делать с {symptom}.",
    "из-за {symptom} я плохо сплю.",
    "моя {symptom} становится хуже с каждым днем.",
    "из-за {symptom} я чувствую усталость каждый день.",
    "этот {symptom} сводит меня с ума.",
    "жить с {symptom} сложно.",
    "весь день испорчен из-за {symptom}.",
    "невозможно сосредоточиться из-за {symptom}.",
    "я начал(a) замечать {symptom} только недавно.",
    "каждое утро начинается с {symptom}.",
    "было бы здорово избавиться от {symptom}.",
    "может ли {symptom} пройти сам по себе?",
    "мои друзья говорят, что {symptom} пройдет.",
    "слышал(a), что {symptom} может быть опасно.",
    "иногда я думаю, что {symptom} исчезло, но потом возвращается.",
    "интернет пишет, что {symptom} может быть разным.",
    "боязно идти к врачу из-за {symptom}.",
    "не знаю, что предпринять против {symptom}.",
    "заметил(a), что {symptom} чаще всего к вечеру.",
    "часто ли {symptom} бывает у других?",
    "существует много советов по поводу {symptom}.",
    "придется привыкать жить с {symptom}.",
    "я сбит(a) с толку из-за {symptom}.",
    "на {symptom} даже лекарства не помогают.",
    "проблемы с {symptom} начались недавно.",
    "возможно, это просто {symptom}.",
    "поговорил(a) со знакомыми, у них тоже было {symptom}.",
    "интересно, это просто {symptom} или что-то более серьезное?",
    "когда начинается {symptom}, мне сразу становится плохо.",
    "было бы замечательно, если бы {symptom} исчезло.",
    "читал(a) форумы о {symptom}, и это пугает.",
    "нужно просто терпеть, пока {symptom} не пройдет.",
    "никогда раньше не было {symptom}.",
    "могу ли я сделать что-то против {symptom}?",
    "есть подозрение, что это {symptom}.",
    "чем может быть вызвано {symptom}?",
]

multiple_symptoms_phrases = [
    "сегодня я обнаружил у себя {symptoms}.",
    "{symptoms} делают мой день невыносимым.",
    "кажется, у меня одновременно есть {symptoms}.",
    "не могу понять, почему вдруг появились {symptoms}.",
    "сегодня я страдаю от {symptoms}.",
    "я чувствую {symptoms}, и не знаю, что с этим делать.",
    "{symptoms} не дают мне покоя.",
    "у меня {symptoms}."
]

def generate_synthetic_text_data(num_samples=100):
    data = []
    for _ in range(num_samples):
        symptom_english, symptom_russian = random.choice(list(symptoms.items()))
        phrase = random.choice(single_symptom_phrases)
        text = phrase.format(symptom=symptom_russian)
        features = {sym: 1 if sym == symptom_english else 0 for sym in symptoms.keys()}
        data.append((text, features))
    for _ in range(num_samples):
        num_symptoms = random.randint(2, 8)  # От 2 до 8 симптомов
        chosen_symptoms = random.sample(list(symptoms.items()), num_symptoms)
        phrase = random.choice(multiple_symptoms_phrases)
        symptoms_texts = [symptom_russian for _, symptom_russian in chosen_symptoms]
        symptoms_text = ', '.join(symptoms_texts)
        text = phrase.format(symptoms=symptoms_text)
        features = {sym: 0 for sym in symptoms.keys()}
        for eng, _ in chosen_symptoms:
            features[eng] = 1
        data.append((text, features))
    
    return data

synthetic_data = generate_synthetic_text_data(num_samples=1000)

In [18]:
df = pd.DataFrame(synthetic_data, columns=['prompt', 'vector'])
print(df.shape)
df.to_csv('syntetic_dataframe.csv', index=False)
df.head()

(2000, 2)


,prompt,vector
0,"заметил(a), что потливость чаще всего к вечеру.","{'vomiting': 0, 'breathlessness': 0, 'sweating..."
1,невозможно сосредоточиться из-за потеря равнов...,"{'vomiting': 0, 'breathlessness': 0, 'sweating..."
2,из-за боль в груди я чувствую усталость каждый...,"{'vomiting': 0, 'breathlessness': 0, 'sweating..."
3,как долго продлится головная боль?,"{'vomiting': 0, 'breathlessness': 0, 'sweating..."
4,в последнее время я замечаю одышка.,"{'vomiting': 0, 'breathlessness': 1, 'sweating..."


# Zero-shot-classification

Zero-shot-classification - это задача прогнозирования класса, который не был замечен моделью во время обучения. Этот метод, использующий предварительно обученную языковую модель. С его помошью мы будем извлекать наличие симптомов в промте пользователя.

В качестве бейзлайна возьмем всемиизвестную модель **Bart** из библиотеки transformers.

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device_map='auto')

# список симптомов (модель обучена на английском, поэтому фичи лучше подавать тоже на английском)
def extract_features(text):  
    features = {
    "vomiting": 0,
    "breathlessness": 0,
    "sweating": 0,
    "headache": 0,
    "chest_pain": 0,
    "dizziness": 0,
    "loss_of_balance": 0,
    "lack_of_concentration": 0
}
    candidate_labels = [' '.join(elem.split('_')) for elem in features.keys()] 
    result = classifier(text, candidate_labels, multi_label=True)

    for label, score in zip(result['labels'], result['scores']):
        if score > 0.5:  # возьмем порог = 0.5
            features[label] = 1
    return features

In [20]:
df['classification_vector'] = df['prompt'].apply(extract_features)
df.head()

,prompt,vector,classification_vector
0,"заметил(a), что потливость чаще всего к вечеру.","{'vomiting': 0, 'breathlessness': 0, 'sweating...","{'vomiting': 0, 'breathlessness': 1, 'sweating..."
1,невозможно сосредоточиться из-за потеря равнов...,"{'vomiting': 0, 'breathlessness': 0, 'sweating...","{'vomiting': 0, 'breathlessness': 0, 'sweating..."
2,из-за боль в груди я чувствую усталость каждый...,"{'vomiting': 0, 'breathlessness': 0, 'sweating...","{'vomiting': 1, 'breathlessness': 1, 'sweating..."
3,как долго продлится головная боль?,"{'vomiting': 0, 'breathlessness': 0, 'sweating...","{'vomiting': 0, 'breathlessness': 0, 'sweating..."
4,в последнее время я замечаю одышка.,"{'vomiting': 0, 'breathlessness': 1, 'sweating...","{'vomiting': 0, 'breathlessness': 0, 'sweating..."


In [21]:
def accuracy(row):
    # моя метрика качества (определяет долю точно проставленных симтомов в векторе)
    correct, symptoms = 0, 0
    for key in row.vector.keys():
        if row.vector[key] == row.classification_vector[key]:
            correct += 1
            symptoms += 1
        else:
            symptoms += 1
    return correct / symptoms

df['accuracy'] = df.apply(accuracy, axis=1)

In [22]:
df.columns = ['user_prompt', 'true_symptoms', 'predicted_symptoms', 'accuracy']
print(df.shape)
df.head()

(2000, 4)


,user_prompt,true_symptoms,predicted_symptoms,accuracy
0,"заметил(a), что потливость чаще всего к вечеру.","{'vomiting': 0, 'breathlessness': 0, 'sweating...","{'vomiting': 0, 'breathlessness': 1, 'sweating...",0.875
1,невозможно сосредоточиться из-за потеря равнов...,"{'vomiting': 0, 'breathlessness': 0, 'sweating...","{'vomiting': 0, 'breathlessness': 0, 'sweating...",0.875
2,из-за боль в груди я чувствую усталость каждый...,"{'vomiting': 0, 'breathlessness': 0, 'sweating...","{'vomiting': 1, 'breathlessness': 1, 'sweating...",0.500
3,как долго продлится головная боль?,"{'vomiting': 0, 'breathlessness': 0, 'sweating...","{'vomiting': 0, 'breathlessness': 0, 'sweating...",0.750
4,в последнее время я замечаю одышка.,"{'vomiting': 0, 'breathlessness': 1, 'sweating...","{'vomiting': 0, 'breathlessness': 0, 'sweating...",0.750


In [25]:
print(f'Средняя точность предсказания симптомов = {df.accuracy.mean()}')

Средняя точность предсказания симптомов = 0.5695


In [26]:
df.to_csv('syntetic_dataframe_final.csv', index=False)

### Подготовим данные для передачи в часть классического ML

In [62]:
data = df.copy()
data = data[['user_prompt', 'predicted_symptoms']]
flags = ("vomiting", "breathlessness", "sweating", "headache", "chest_pain", "dizziness", "loss_of_balance", "lack_of_concentration")

for flag in flags:
    data[flag] = data.apply(lambda row: 1 if dict(row['predicted_symptoms'])[flag] == 1 else 0, axis=1)

data = data.drop(columns=['predicted_symptoms'])
data.head()

,user_prompt,vomiting,breathlessness,sweating,headache,chest_pain,dizziness,loss_of_balance,lack_of_concentration
0,"заметил(a), что потливость чаще всего к вечеру.",0,1,1,0,0,0,0,0
1,невозможно сосредоточиться из-за потеря равнов...,0,0,0,0,0,0,0,0
2,из-за боль в груди я чувствую усталость каждый...,1,1,0,0,0,1,0,0
3,как долго продлится головная боль?,0,0,0,0,0,1,0,0
4,в последнее время я замечаю одышка.,0,0,0,0,0,1,0,0
